In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os
import sys
# Needed for the model class to work inside a Notebook.
os.environ["COBAYA_NOMPI"] = "True"

from cobaya.yaml          import yaml_load_file
from cobaya.samplers.mcmc import plot_progress
from cobaya.model         import get_model
from cobaya.samplers.mcmc import plot_progress
#
from getdist.mcsamples    import MCSamplesFromCobaya
from getdist.mcsamples    import loadMCSamples
import getdist.plots      as     gdplt
#
import os

In [3]:
# Add everything to our paths.
repo = "/global/cscratch1/sd/sfschen/CobayaBAO_DESI/"
for codedir in ["lss_likelihood"]:
    sys.path.append(repo+codedir)

In [4]:
samp = 'cubic'
info = yaml_load_file("recsym_cubic_pkells.yaml")
print(info['params'].keys())
print(info['output'])

dict_keys(['apar', 'aperp', 'B1_cubic', 'F_cubic', 'M0_cubic', 'M1_cubic', 'M2_cubic', 'M3_cubic', 'M4_cubic', 'Q0_cubic', 'Q1_cubic', 'Q2_cubic', 'Q3_cubic', 'Q4_cubic'])
chains/recsym_cubic_pkells


In [5]:
params = {'apar': 0.98465368, 'aperp': 1.0007494000000001, 'B1_cubic': 0.95585343, 'F_cubic': 1.0216138000000001, 'M0_cubic': 22.603372, 'M1_cubic': -753.61969, 'M2_cubic': -781.5875, 'M3_cubic': 549.61425, 'M4_cubic': -79.843745, 'Q0_cubic': 86.363784, 'Q1_cubic': -3151.9162, 'Q2_cubic': 1539.9194, 'Q3_cubic': -1403.7662, 'Q4_cubic': 274.9908}

In [6]:
model = get_model(info)
print(list(model.parameterization.sampled_params()))

[model] *WARNING* Ignored blocks/options: ['sampler', 'output', 'stop_on_error']
(10.556307852555234, 9.45499094916356, 10.565115328400125, 13.777685164072894, 0.03123000341125935)
['apar', 'aperp', 'B1_cubic', 'F_cubic', 'M0_cubic', 'M1_cubic', 'M2_cubic', 'M3_cubic', 'M4_cubic', 'Q0_cubic', 'Q1_cubic', 'Q2_cubic', 'Q3_cubic', 'Q4_cubic']


In [7]:
model.logposterior(params)
lik = model.likelihood['recsym_likelihood.RecSymLikelihood']

tt = lik.bao_predict(samp)
tt = lik.bao_observe(tt)

In [8]:
# Compute templates:
lik_str = 'recsym_likelihood.RecSymLikelihood'
pars_list = params.keys()

In [9]:
err = 0.005
dPs = {}

for param_str in pars_list:
    param = params[param_str]
    
    pars_plus = params.copy()
    pars_plus[param_str] = param + err
    
    pars_minus = params.copy()
    pars_minus[param_str] = param - err

    model.logposterior(pars_plus)
    obs_plus = 1.0 * model.likelihood[lik_str].thy_obs
        
    model.logposterior(pars_minus)
    obs_minus = 1.0 * model.likelihood[lik_str].thy_obs
        
    dPs[param_str] = (obs_plus - obs_minus)/(2*err)

In [10]:
# Dot into covariance to make Fisher matrix
Fij = np.zeros( (len(params.keys()),)*2 )
    
for ii, param_ii in enumerate(pars_list):
    for jj, param_jj in enumerate(pars_list):
            
        Fij[ii,jj] = np.dot(dPs[param_ii],\
                                np.dot(model.likelihood[lik_str].cinv,\
                                       dPs[param_jj]))

In [11]:
np.diag(np.sqrt(np.linalg.inv(Fij)))

<ipython-input-11-7dce64f41c76>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij)))


array([8.67114938e-03, 5.57118956e-03, 5.37498668e-02, 1.32336459e-01,
       2.07004665e+01, 6.13663732e+02, 3.72008066e+02, 1.88042389e+02,
       3.30221097e+01, 5.41332828e+01, 1.62900508e+03, 9.40961637e+02,
       4.50407082e+02, 7.81874993e+01])

In [12]:
# Add Priors
iis =  (np.arange(len(pars_list) ) ) > 3 # no apar, aperp, B1, F
prior_inv_sigmas = iis * 1./100000**2

Fij_with_prior = Fij + np.diag(prior_inv_sigmas)

In [13]:
np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))

<ipython-input-13-857f7eb08c5c>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))


array([8.67107934e-03, 5.57117517e-03, 5.37467105e-02, 1.32319466e-01,
       2.06997219e+01, 6.13635808e+02, 3.71996411e+02, 1.88040153e+02,
       3.30218479e+01, 5.41262529e+01, 1.62874823e+03, 9.40852164e+02,
       4.50384486e+02, 7.81847556e+01])

In [14]:
corr_mat = np.linalg.inv(Fij_with_prior)/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[None,:]

<ipython-input-14-9a6d4c649bc8>:1: RuntimeWarning: invalid value encountered in sqrt
  corr_mat = np.linalg.inv(Fij_with_prior)/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[None,:]


In [15]:
corr_mat[:4,:4]

array([[ 1.        , -0.42947265,  0.17054847, -0.23576381],
       [-0.42947265,  1.        , -0.21788569,  0.10613416],
       [ 0.17054847, -0.21788569,  1.        , -0.7385502 ],
       [-0.23576381,  0.10613416, -0.7385502 ,  1.        ]])

In [16]:
np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))

<ipython-input-16-6b31f6b9f5ce>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))


array([0.00639544, 0.00456102, 0.01033815, 0.02015148])

In [65]:
np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:2,:2])))

<ipython-input-65-2c22f3d3ae50>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:2,:2])))


array([0.00429801, 0.0028842 ])

In [66]:
corr_mat = np.linalg.inv(Fij_with_prior[:4,:4])/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[None,:]

<ipython-input-66-800a0b0b04ac>:1: RuntimeWarning: invalid value encountered in sqrt
  corr_mat = np.linalg.inv(Fij_with_prior[:4,:4])/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[None,:]


In [67]:
corr_mat[:4,:4]

array([[ 1.        , -0.36600074,  0.48918734, -0.90307345],
       [-0.36600074,  1.        , -0.93029699,  0.59470038],
       [ 0.48918734, -0.93029699,  1.        , -0.74390488],
       [-0.90307345,  0.59470038, -0.74390488,  1.        ]])